Writing Algorithm:

In [1]:
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
import pandas_datareader as pdr
    P_VATHRESHOLD = 0.05
    HURST= 0.5
    MIN_MEAN = 12
    TRADING = 1
    EXIT= 0
    STOP = 2
    RETURN_ON_BOTH_LEGS = False
    WAITING = 30
    TRADING = 253
    #percentage of transaction costs
    TC = 0

def get_data(tickers, start_date, end_date):
    """
    Retrieve daily price data for the specified tickers and date range.
    
    Parameters
    ----------
    tickers : list of str
        A list of ticker symbols for the securities to retrieve data for.
    start_date : str
        The start date for the data in the format 'YYYY-MM-DD'.
    end_date : str
        The end date for the data in the format 'YYYY-MM-DD'.
        
    Returns
    -------
    data : pandas DataFrame
        A DataFrame containing the daily price data for the securities.
    """
    data = pdr.get_data_yahoo(tickers, start_date, end_date)['Adj Close']
    return data

def select_pairs(data):
    """
    Select all possible pairs of securities from the data.
    
    Parameters
    ----------
    data : pandas DataFrame
        A DataFrame containing the daily price data for the securities.
        
    Returns
    -------
    pairs : list of tuples
        A list of tuples, where each tuple contains the ticker symbols for a pair of securities.
    """
    tickers = data.columns
    pairs = [(t1, t2) for i, t1 in enumerate(tickers) for j, t2 in enumerate(tickers) if i < j]
    return pairs

def find_cointegrated_pairs(data):
    """
    Find pairs of securities from the data that are cointegrated.
    
    Parameters
    ----------
    data : pandas DataFrame
        A DataFrame containing the daily price data for the securities.
        
    Returns
    -------
    cointegrated_pairs : list of tuples
        A list of tuples, where each tuple contains the ticker symbols for a pair of cointegrated securities.
    """
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = sm.tsa.stattools.coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((keys[i], keys[j]))
    return pairs

def pairs_trading(data, pairs, z_score_threshold=1.0):
    """
    Execute a pairs trading strategy on the data using the specified pairs and z-score threshold.
    
    Parameters
    ----------
    data : pandas DataFrame
        A DataFrame containing the daily price data for the securities.
    pairs : list of tuples
        A list of tuples, where each tuple contains the ticker symbols for a pair of securities.
    z_score_threshold : float, optional
        The z-score threshold to use for the pairs trading strategy.
    """
    for pair in pairs:
        S1 = data[pair[0]]
        S2 = data[pair[1]]
        # Calculate the spread
        spread = S1 - S2
        # Calculate the z-score of the spread
        z_score = (spread - spread.mean()) / spread.std()
        # Sell the first security and buy the second security if the z-score is above the threshold
        if z_score > z_score_threshold:
            print(f'Sell {pair[0]} and buy {pair[1]}')
        # Buy the first security and sell the second security if the z-score is below the threshold
        elif z_score < -z_score_threshold:
            print(f'Buy {pair[0]} and sell {pair[1]}')